In [156]:
import pandas as pd
import seaborn as sb

#df = pd.read_csv('forestfires.csv')
df_portugal = pd.read_csv('data/Portugal_Dataset.csv')
df_portugal
#sb.heatmap(df.corr(), annot = True, fmt='.1g')


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.00
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.00
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.00
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.00
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.00


In [157]:
#Creates a dataframe containing california counties and their relative biomass(dead & alive) in tons
#4 counties unaccounted for in dead biomass so it's assumed that these areas have negligible dead biomass
df61 = pd.read_csv('data/data61.txt', delim_whitespace=True)
df61 = df61[['County', 'All_Total']]
df61 = df61.rename(columns={'All_Total': "Alive_Biomass_Tons"})
df62 = pd.read_csv('data/data62.txt', delim_whitespace=True)
df62 = df62[['County', 'All_Total']]
df62 = df62.rename(columns={'All_Total': "Dead_Biomass_Tons"})
for county in df61['County']:
    if county not in df62['County'].values:
        df62 = df62.append({'County': county,"Dead_Biomass_Tons": 0}, ignore_index=True)
df62 = df62.sort_values(by ='County', ascending = 1)
dfMass = pd.merge(df61,df62, on = ['County'])

for county in dfMass['County']:
    if "_" in county:
        fix = county.replace("_", " ")
        dfMass = dfMass.replace(county, fix)
dfMass

,County,Alive_Biomass_Tons,Dead_Biomass_Tons
0,Alameda,"5,363.1",205.4
1,Alpine,"21,755.0","3,009.8"
2,Amador,"13,987.8","1,506.8"
3,Butte,"46,750.7","3,181.1"
4,Calaveras,"27,078.1","1,732.0"
5,Colusa,"5,376.2",363.1
6,Contra Costa,"2,799.0",10.4
7,Del Norte,"81,591.3","8,065.7"
8,El Dorado,"73,636.9","7,813.7"
9,Fresno,"80,129.4","8,425.4"


In [158]:
#this function returns a dictionary containing the average weather attributes in over a fires burn time
#"start" and "end" should take the form passed from the columns: Started and Extingushed (ex. parameter: 2019-01-04T09:33:00Z)
#"stn" is the station ID
def getWeather(start, end, stn):
    last_days = {1:31, 2:28, 3:31, 4:30,5:31 ,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    year = int(start[0:4])
    month = int(start[5:7])
    day = int(start[8:10])
    if(day == 29 and month == 2):#incase of leap year
        day = 1
        month = 3
    Emonth = int(end[5:7])
    Eday = int(end[8:10])
    if(Eday == 29 and Emonth == 2):#incase of leap year
        Eday = 1
        Emonth = 3
    days = 0
    aRain = 0
    aTemp = 0
    aHum = 0 
    aWind = 0
    Eday += 1
    while(not ((day is Eday) and (month is Emonth))):
        dayWeather = df_cal_weather.loc[(df_cal_weather['Stn Id'] == stn) & (df_cal_weather['Month'] == month) & (df_cal_weather['Day'] == day) & (df_cal_weather['Year'] == year)]
        if(dayWeather.empty):
            print("No weather data for %d/%d/%d at station: %d" %(month, day, year, stn))
            return {}
        aRain += dayWeather['Precip (in)'].values
        aTemp += dayWeather['Avg Air Temp (F)'].values
        aHum  += dayWeather['Avg Rel Hum (%)'].values
        aWind += dayWeather['Avg Wind Speed (mph)'].values
        days += 1
        day += 1
        
        if(day > last_days.get(month)):
            day = 1
            month += 1
            if(month == 13):
                month = 1
                year += 1 
    aRain /= days
    aTemp /= days
    aHum /= days
    aWind /= days
    avg_weather= {"Days Burned": days,"Precipitation" : round(aRain[0], 6), "Temperature" : round(aTemp[0],6), "Humidity" : round(aHum[0],6), "Wind Speed" : round(aWind[0],6)}
    
    return avg_weather

In [159]:
# Remove irrelevant columns
# Location may be irrelevant because all are xy coordinates for Montesinho park
columns_to_keep_portugal = [
    'month', 
    'day', 
    'temp', 
    'RH',  # relative humidity
    'wind',  # wind speed
    'rain',  # rain amount
    'area'  # burned area
]

df_portugal = df_portugal[columns_to_keep_portugal]
df_portugal.shape


(517, 7)

In [160]:
# Removing all missing data
df_portugal = df_portugal.dropna()
print(df_portugal.shape)

(517, 7)


In [161]:
# Show updated dataset
df_portugal
df_portugal.to_csv('data/Portugal_Dataset_clean.csv')

In [162]:
import pandas as pd
import seaborn as sb

df_cal_weather = pd.read_csv('data/California_Weather.csv' )

#df_cal_weather

In [163]:
# Remove irrelevant columns
columns_to_keep_cal_weather = [
    'Stn Id',
    'Stn Name',
    'CIMIS Region', 
    'Date', 
    'Precip (in)',
    'Avg Air Temp (F)',
    'Avg Rel Hum (%)',
    'Avg Wind Speed (mph)',
    'Wind Run (miles)'
]

df_cal_weather = df_cal_weather[columns_to_keep_cal_weather]
df_cal_weather.shape

(128125, 9)

In [164]:
# Removing all missing data
df_cal_weather = df_cal_weather.dropna()
print(df_cal_weather.shape)

(128112, 9)


In [165]:
# Combine the data from California Weather and California Fire
# Combine based on location and Date

# Split the Date column from California Weather into month / year
df_cal_weather['Month'] = df_cal_weather['Date'].str.split('/').str[0].astype('int')
df_cal_weather['Year'] = df_cal_weather['Date'].str.split('/').str[2].astype('int')
df_cal_weather['Day'] = df_cal_weather['Date'].str.split('/').str[1].astype('int')



df_cal_weather

,Stn Id,Stn Name,CIMIS Region,Date,Precip (in),Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),Wind Run (miles),Month,Year,Day
0,2,FivePoints,San Joaquin Valley,1/1/2018,0.00,47.8,65.0,3.3,78.3,1,2018,1
1,2,FivePoints,San Joaquin Valley,1/2/2018,0.00,47.2,67.0,3.1,74.5,1,2018,2
2,2,FivePoints,San Joaquin Valley,1/3/2018,0.00,49.9,68.0,4.5,107.5,1,2018,3
3,2,FivePoints,San Joaquin Valley,1/4/2018,0.01,56.8,74.0,5.8,140.2,1,2018,4
4,2,FivePoints,San Joaquin Valley,1/5/2018,0.00,59.8,72.0,4.2,101.4,1,2018,5
5,2,FivePoints,San Joaquin Valley,1/6/2018,0.04,55.2,90.0,6.7,162.0,1,2018,6
6,2,FivePoints,San Joaquin Valley,1/7/2018,0.00,56.4,74.0,3.5,83.3,1,2018,7
7,2,FivePoints,San Joaquin Valley,1/8/2018,0.38,55.6,92.0,5.1,122.0,1,2018,8
8,2,FivePoints,San Joaquin Valley,1/10/2018,0.00,54.1,75.0,5.7,137.6,1,2018,10
9,2,FivePoints,San Joaquin Valley,1/11/2018,0.00,52.9,80.0,2.6,61.8,1,2018,11


In [166]:
# Merge Cal Weather with CIMIS data
import glob

files_list = glob.glob("data/CIMISReports/new/*.csv")
#print(files_list)

column_headers = ['County', 'Month', 'Precip (in)', 'Avg Air Temp (F)', 'Avg Rel Hum (%)', 'Avg Wind Speed (mph)' ]

for filename in files_list:
    end_county_name = filename.index('.')
    county_name = filename[ 22:end_county_name ]
    data = {}

        
    # Initialize columns
    for header in column_headers:
        data[header] = []

    # Read each file
    df_temp = pd.read_csv(filename)
    df_cal_weather = pd.concat([df_cal_weather, df_temp])
    

C:\Users\yzh\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [167]:
df_cal_weather

,Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),CIMIS Region,County,Date,Day,Month,Precip (in),Stn Id,Stn Name,Wind Run (miles),Year
0,47.8,65,3.3,San Joaquin Valley,NaN,1/1/2018,1.0,1,0,2.0,FivePoints,78.3,2018
1,47.2,67,3.1,San Joaquin Valley,NaN,1/2/2018,2.0,1,0,2.0,FivePoints,74.5,2018
2,49.9,68,4.5,San Joaquin Valley,NaN,1/3/2018,3.0,1,0,2.0,FivePoints,107.5,2018
3,56.8,74,5.8,San Joaquin Valley,NaN,1/4/2018,4.0,1,0.01,2.0,FivePoints,140.2,2018
4,59.8,72,4.2,San Joaquin Valley,NaN,1/5/2018,5.0,1,0,2.0,FivePoints,101.4,2018
5,55.2,90,6.7,San Joaquin Valley,NaN,1/6/2018,6.0,1,0.04,2.0,FivePoints,162.0,2018
6,56.4,74,3.5,San Joaquin Valley,NaN,1/7/2018,7.0,1,0,2.0,FivePoints,83.3,2018
7,55.6,92,5.1,San Joaquin Valley,NaN,1/8/2018,8.0,1,0.38,2.0,FivePoints,122.0,2018
8,54.1,75,5.7,San Joaquin Valley,NaN,1/10/2018,10.0,1,0,2.0,FivePoints,137.6,2018
9,52.9,80,2.6,San Joaquin Valley,NaN,1/11/2018,11.0,1,0,2.0,FivePoints,61.8,2018


In [168]:
#import CIMIS station data, drop Stn Name(duplicate with CA weather) and Status(irrelevant)
df_cimis = pd.read_csv('data/cimis_stations.csv', names=['Stn Id', 'Stn Name', 'Counties', 'Status'])

df_cimis = df_cimis[['Stn Id','Counties']]

#combine CIMIS data with CA weather data (CIMIS data has counties that are in the same format as in CA fire)
df_weather_with_counties = pd.merge(df_cal_weather,df_cimis, on = ['Stn Id'])



In [169]:
# Show updated dataset
"""
df_cal_weather.to_csv('data/California_Weather_clean.csv')
df_cal_weather
"""
df_weather_with_counties = df_weather_with_counties.drop(columns = ['County'])
df_weather_with_counties = df_weather_with_counties.rename(columns={"Counties": "County"})
df_weather_with_counties.to_csv('data/California_Weather_clean.csv')
df_weather_with_counties

,Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),CIMIS Region,Date,Day,Month,Precip (in),Stn Id,Stn Name,Wind Run (miles),Year,County
0,47.8,65,3.3,San Joaquin Valley,1/1/2018,1.0,1,0,2.0,FivePoints,78.3,2018,Fresno
1,47.2,67,3.1,San Joaquin Valley,1/2/2018,2.0,1,0,2.0,FivePoints,74.5,2018,Fresno
2,49.9,68,4.5,San Joaquin Valley,1/3/2018,3.0,1,0,2.0,FivePoints,107.5,2018,Fresno
3,56.8,74,5.8,San Joaquin Valley,1/4/2018,4.0,1,0.01,2.0,FivePoints,140.2,2018,Fresno
4,59.8,72,4.2,San Joaquin Valley,1/5/2018,5.0,1,0,2.0,FivePoints,101.4,2018,Fresno
5,55.2,90,6.7,San Joaquin Valley,1/6/2018,6.0,1,0.04,2.0,FivePoints,162.0,2018,Fresno
6,56.4,74,3.5,San Joaquin Valley,1/7/2018,7.0,1,0,2.0,FivePoints,83.3,2018,Fresno
7,55.6,92,5.1,San Joaquin Valley,1/8/2018,8.0,1,0.38,2.0,FivePoints,122.0,2018,Fresno
8,54.1,75,5.7,San Joaquin Valley,1/10/2018,10.0,1,0,2.0,FivePoints,137.6,2018,Fresno
9,52.9,80,2.6,San Joaquin Valley,1/11/2018,11.0,1,0,2.0,FivePoints,61.8,2018,Fresno


In [170]:
print(df_weather_with_counties['County'].unique())
print(df_weather_with_counties.columns)

['Fresno' 'Kern' 'Yolo' 'Butte' 'El Dorado' 'Kings' 'Inyo' 'Imperial'
 'Shasta' 'Riverside' 'Contra Costa' 'San Luis Obispo' 'Merced' 'Lassen'
 'Santa Barbara' 'San Joaquin' 'Stanislaus' 'Orange' 'Napa' 'Los Angeles'
 'Sonoma' 'Yuba' 'Tulare' 'Modoc' 'Siskiyou' 'Santa Cruz' 'Mendocino'
 'Monterey' 'San Bernardino' 'Solano' 'San Benito' 'Sacramento'
 'San Diego' 'Ventura' 'Marin' 'Alameda' 'Placer' 'Santa Clara' 'Tehama'
 'Amador' 'Sutter' 'Colusa' 'San Mateo' 'Alpine' 'Humboldt']
Index(['Avg Air Temp (F)', 'Avg Rel Hum (%)', 'Avg Wind Speed (mph)',
       'CIMIS Region', 'Date', 'Day', 'Month', 'Precip (in)', 'Stn Id',
       'Stn Name', 'Wind Run (miles)', 'Year', 'County'],
      dtype='object')


In [171]:
import pandas as pd
import seaborn as sb

df_cal_fire = pd.read_csv('data/California_Fire_Incidents.csv')
#df_cal_fire


In [172]:
# Remove irrelevant columns
columns_to_keep_cal_fire = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'Counties', 
    'Extinguished',
    'Location',  
    'Started'
]

df_cal_fire = df_cal_fire[columns_to_keep_cal_fire]
df_cal_fire.shape

(1636, 7)

In [173]:
# Removing all missing data
df_cal_fire = df_cal_fire.dropna()
print(df_cal_fire.shape)

(1577, 7)


In [174]:
# Show updated dataset
df_cal_fire
df_cal_fire.to_csv('data/California_Fire_Incidents_clean.csv')

In [175]:
# Split the Started column from California Weather into month / year
df_cal_fire['year'] = df_cal_fire['Started'].str.split('-').str[0].astype('int')
df_cal_fire['month'] = df_cal_fire['Started'].str.split('-').str[1].astype('int')

# Rename month/year columns
df_cal_fire = df_cal_fire.rename(columns={"Counties": "County"})
df_cal_fire = df_cal_fire.rename(columns={"month": "Month", "year": "Year"})

df_cal_fire


,AcresBurned,ArchiveYear,CalFireIncident,County,Extinguished,Location,Started,Year,Month
0,257314.0,2013,True,Tuolumne,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,2013-08-17T15:25:00Z,2013,8
1,30274.0,2013,True,Los Angeles,2013-06-08T18:30:00Z,Angeles National Forest,2013-05-30T15:28:00Z,2013,5
2,27531.0,2013,True,Riverside,2013-07-30T18:00:00Z,Hwy 243 & Hwy 74 near Mountain Center,2013-07-15T13:43:00Z,2013,7
3,27440.0,2013,False,Placer,2013-08-30T08:00:00Z,"Deadwood Ridge, northeast of Foresthill",2013-08-10T16:30:00Z,2013,8
4,24251.0,2013,True,Ventura,2013-05-11T06:30:00Z,Southbound Highway 101 at Camarillo Springs Ro...,2013-05-02T07:01:00Z,2013,5
5,22992.0,2013,False,Fresno,2013-09-24T20:15:00Z,Seven miles north of Big Creek,2013-07-22T22:15:00Z,2013,7
6,20292.0,2013,True,Riverside,2013-08-12T18:00:00Z,"Poppet Flats Rd near Hwy 243, south of Banning",2013-08-07T14:05:00Z,2013,8
7,14754.0,2013,False,Siskiyou,2013-08-31T06:45:00Z,"North Fork of the Salmon River, West of Sawyer...",2013-07-31T22:00:00Z,2013,7
8,12503.0,2013,False,Humboldt,2013-08-12T12:00:00Z,Tish Tang Ridge east of Hoopa Valley Reservation,2013-08-10T11:40:00Z,2013,8
9,11429.0,2013,True,Tehama,2013-08-29T16:45:00Z,"Near Deer Creek, 12 miles east of Los Molinos.",2013-08-23T14:15:00Z,2013,8


In [176]:
# FIXME

#combine matching entries of CA fire and CA weather by County, month, and year
df_CA_merged = pd.merge(df_cal_fire,df_weather_with_counties, how = 'inner', on = ['County','Month','Year'])

df_CA_merged


,AcresBurned,ArchiveYear,CalFireIncident,County,Extinguished,Location,Started,Year,Month,Avg Air Temp (F),Avg Rel Hum (%),Avg Wind Speed (mph),CIMIS Region,Date,Day,Precip (in),Stn Id,Stn Name,Wind Run (miles)
0,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,82.7,52,4.8,Sacramento Valley,7/1/2018,1.0,0,250.0,Williams,114.4
1,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,75.8,67,5,Sacramento Valley,7/2/2018,2.0,0,250.0,Williams,119.4
2,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,73.9,63,5,Sacramento Valley,7/3/2018,3.0,0,250.0,Williams,119.9
3,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,67.6,72,5.6,Sacramento Valley,7/4/2018,4.0,0,250.0,Williams,135.6
4,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,71,65,3.8,Sacramento Valley,7/5/2018,5.0,0,250.0,Williams,90.6
5,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,74,67,2.9,Sacramento Valley,7/6/2018,6.0,0,250.0,Williams,70.7
6,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,76.2,63,3.4,Sacramento Valley,7/7/2018,7.0,0,250.0,Williams,81.7
7,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,74.7,66,4.1,Sacramento Valley,7/8/2018,8.0,0,250.0,Williams,99.3
8,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,75.4,59,3.9,Sacramento Valley,7/9/2018,9.0,0,250.0,Williams,92.8
9,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,"off Highway 20 near Potter Valley, northeast o...",2018-07-27T12:05:00Z,2018,7,79.6,50,3.7,Sacramento Valley,7/10/2018,10.0,0,250.0,Williams,88.4


In [177]:
# Show updated dataset
df_CA_merged
df_CA_merged.to_csv('data/California_Merged_clean.csv')

In [178]:
getWeather('2018-07-27T12:05:00Z','2019-01-04T09:33:00Z', 250)

{'Days Burned': 162,
 'Precipitation': 0.019444,
 'Temperature': 60.873457,
 'Humidity': 64.654321,
 'Wind Speed': 4.47284}

In [179]:
"""
# Get the unique Regions/Counties of weather/fire and use 'str.replace()' to merge identical values

# Cleaning up Cal_Fire Counties to match Cal_weather Regions Choices

# San Francisco Bay
sf_bay = ['Alameda', 'Amador', 'Marin', 'Napa', 'San Mateo', 'Santa Clara', 'Sonoma']
# ?
cent_cal = ['Fresno', 'Monterey','San Joaquin Valley']
# Sacramento Valley
sac_valley = ['Butte', 'Sacramento Valley', 'Sutter', 'Yuba']
# North Coast Valleys
north_coast = ['Humboldt','Mendocino']
# ?
in_so_cal = ['Kern', 'San Bernardino']
# Los Angeles Basin
la_basin = ['Los Angeles']
# ?
great_basin = ['Modoc']
# Sierra Foothill
sierra_nv = ['Nevada', 'Sierra', 'State of Nevada']
# South Coast Valleys
so_coast = ['Orange', 'San Diego']
# Central Coast Valleys
cent_coast = ['San Luis Obispo', 'Santa Barbara', 'Santa Cruz']
# ?
shasta_cas = [ 'Shasta', 'Trinity']

# Unknown:
# Alpine, Calaveras, Colusa, Contra Costa, Del Norte, El Dorado, Glenn, Inyo, Kings, Lake, Lassen, Madera
# Mariposa, Merced, Mexico, Mono, Placer, Plumas, Riverside, San Benito, Siskiyou, Solano, Stanislaus
# State of Oregon, Tehama, Tulare, Tuolumne, Ventura, Yolo,


df_cal_fire['Counties'] = df_cal_fire['Counties'].str.replace('Alameda', 'San Francisco Bay')

regions_list = {'SanFranciscoBay': sf_bay, 'CentralCal': cent_cal, 'SacramentoValley': sac_valley, 
                'NorthCoastValleys': north_coast, 'InlandSoCal': in_so_cal, 'LosAngelesBasin': la_basin, 
                'GreatBasin': great_basin, 'SierraFoothill': sierra_nv, 'SouthCoastValleys': so_coast, 
                'CentralCoastValleys': cent_coast, 'ShastaCascade': shasta_cas}

for region_name in regions_list:
    for county in regions_list[region_name]:
        # FIXME
        df_cal_fire['Counties'] = df_cal_fire['Counties'].str.replace(county, region_name)

unique_regions = (df_cal_weather['CIMIS Region'].unique())
print(sorted(unique_regions))
unique_counties = (df_cal_fire['Counties'].unique())
print(sorted(unique_counties))

"""

"\n# Get the unique Regions/Counties of weather/fire and use 'str.replace()' to merge identical values\n\n# Cleaning up Cal_Fire Counties to match Cal_weather Regions Choices\n\n# San Francisco Bay\nsf_bay = ['Alameda', 'Amador', 'Marin', 'Napa', 'San Mateo', 'Santa Clara', 'Sonoma']\n# ?\ncent_cal = ['Fresno', 'Monterey','San Joaquin Valley']\n# Sacramento Valley\nsac_valley = ['Butte', 'Sacramento Valley', 'Sutter', 'Yuba']\n# North Coast Valleys\nnorth_coast = ['Humboldt','Mendocino']\n# ?\nin_so_cal = ['Kern', 'San Bernardino']\n# Los Angeles Basin\nla_basin = ['Los Angeles']\n# ?\ngreat_basin = ['Modoc']\n# Sierra Foothill\nsierra_nv = ['Nevada', 'Sierra', 'State of Nevada']\n# South Coast Valleys\nso_coast = ['Orange', 'San Diego']\n# Central Coast Valleys\ncent_coast = ['San Luis Obispo', 'Santa Barbara', 'Santa Cruz']\n# ?\nshasta_cas = [ 'Shasta', 'Trinity']\n\n# Unknown:\n# Alpine, Calaveras, Colusa, Contra Costa, Del Norte, El Dorado, Glenn, Inyo, Kings, Lake, Lassen, Madera

In [180]:

def getWeather(start, end, stn):
    last_days = {1:31, 2:28, 3:31, 4:30,5:31 ,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    year = int(start[0:4])
    month = int(start[5:7])
    day = int(start[8:10])

    if(day == 29 and month == 2):#incase of leap year
        day = 1
        month = 3
    Emonth = int(end[5:7])
    Eday = int(end[8:10])
    if(Eday == 29 and Emonth == 2):#increase of leap year
        Eday = 1
        Emonth = 3
    days = 0
    aRain = 0
    aTemp = 0
    aHum = 0 
    aWind = 0
    exist_day = 0
    while True:
        dayWeather = df_cal_weather.loc[(df_cal_weather['Stn Id'] == stn) & (df_cal_weather['Month'] == month) & (df_cal_weather['Day'] == day) & (df_cal_weather['Year'] == year)]
        if(dayWeather.empty):
            print("No weather data for %d/%d/%d at station: %d" %(month, day, year, stn))
        else:
            try: 
                aRain += float(dayWeather['Precip (in)'].values)
                aTemp += float(dayWeather['Avg Air Temp (F)'].values)
                aHum  += float(dayWeather['Avg Rel Hum (%)'].values)
                aWind += float(dayWeather['Avg Wind Speed (mph)'].values)

                exist_day += 1
            except:
                pass
        
        days +=1


        if (day == Eday) and (month == Emonth):
            break
            
        day += 1
        
        if (days > 500):
            return
        
        
        if(day > last_days.get(month)):
            day = 1
            month += 1
            if(month == 13):
                month = 1
                year += 1 
        
    
    if exist_day != 0:
        aRain /= exist_day
        aTemp /= exist_day
        aHum /= exist_day
        aWind /= exist_day
    else:
        exist_day+=1
        aRain /= exist_day
        aTemp /= exist_day
        aHum /= exist_day
        aWind /= exist_day
        
    avg_weather= {"Days Burned": days,"Precipitation" : round(aRain, 6), "Temperature" : round(aTemp,6), "Humidity" : round(aHum,6), "Wind Speed" : round(aWind,6)}
    
    return avg_weather

In [181]:
Important_features = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'County', 
    'Extinguished',
    'Started',
    "Stn Id",
    "Precip (in)",
    "CIMIS Region"
]
Weather_feature = [
   'Days Burned', 'Precipitation', 'Temperature', 'Humidity', 'Wind Speed',
]

In [182]:
Unique_fire_location = df_CA_merged.Location.unique()

CA_fire_actual_DF = {}
for i in Important_features:
    CA_fire_actual_DF[i] = []

for i in Weather_feature:
    CA_fire_actual_DF[i] = []
    
for i in range(len(Unique_fire_location)):
    current_unique_event = df_CA_merged.loc[df_CA_merged['Location'] == Unique_fire_location[i]].iloc[0]
    for j in Important_features:
        CA_fire_actual_DF[j].append(current_unique_event[j])

  
    AvgWeather = getWeather(current_unique_event["Started"],current_unique_event["Extinguished"],current_unique_event["Stn Id"])

    
    for j in Weather_feature:
        CA_fire_actual_DF[j].append(AvgWeather[j])
        
        
for i in CA_fire_actual_DF:
    print(len(CA_fire_actual_DF[i]))

No weather data for 7/19/2018 at station: 250
No weather data for 7/26/2018 at station: 250
No weather data for 7/31/2018 at station: 106
No weather data for 8/1/2018 at station: 106
No weather data for 8/2/2018 at station: 106
No weather data for 8/3/2018 at station: 106
No weather data for 8/4/2018 at station: 106
No weather data for 8/5/2018 at station: 106
No weather data for 8/6/2018 at station: 106
No weather data for 8/7/2018 at station: 106
No weather data for 8/8/2018 at station: 106
No weather data for 8/9/2018 at station: 106
No weather data for 8/10/2018 at station: 106
No weather data for 8/11/2018 at station: 106
No weather data for 8/12/2018 at station: 106
No weather data for 8/13/2018 at station: 106
No weather data for 8/14/2018 at station: 106
No weather data for 8/15/2018 at station: 106
No weather data for 8/16/2018 at station: 106
No weather data for 8/17/2018 at station: 106
No weather data for 8/18/2018 at station: 106
No weather data for 8/19/2018 at station: 1

No weather data for 9/13/2018 at station: 106
No weather data for 9/14/2018 at station: 106
No weather data for 9/15/2018 at station: 106
No weather data for 9/16/2018 at station: 106
No weather data for 9/17/2018 at station: 106
No weather data for 9/18/2018 at station: 106
No weather data for 9/19/2018 at station: 106
No weather data for 9/20/2018 at station: 106
No weather data for 9/21/2018 at station: 106
No weather data for 9/22/2018 at station: 106
No weather data for 9/23/2018 at station: 106
No weather data for 9/24/2018 at station: 106
No weather data for 9/25/2018 at station: 106
No weather data for 9/26/2018 at station: 106
No weather data for 9/27/2018 at station: 106
No weather data for 9/28/2018 at station: 106
No weather data for 9/29/2018 at station: 106
No weather data for 9/30/2018 at station: 106
No weather data for 10/1/2018 at station: 106
No weather data for 10/2/2018 at station: 106
No weather data for 10/3/2018 at station: 106
No weather data for 10/4/2018 at s

No weather data for 12/5/2018 at station: 106
No weather data for 12/6/2018 at station: 106
No weather data for 12/7/2018 at station: 106
No weather data for 12/8/2018 at station: 106
No weather data for 12/9/2018 at station: 106
No weather data for 12/10/2018 at station: 106
No weather data for 12/11/2018 at station: 106
No weather data for 12/12/2018 at station: 106
No weather data for 12/13/2018 at station: 106
No weather data for 12/14/2018 at station: 106
No weather data for 12/15/2018 at station: 106
No weather data for 12/16/2018 at station: 106
No weather data for 12/17/2018 at station: 106
No weather data for 12/18/2018 at station: 106
No weather data for 12/19/2018 at station: 106
No weather data for 12/20/2018 at station: 106
No weather data for 12/21/2018 at station: 106
No weather data for 12/22/2018 at station: 106
No weather data for 12/23/2018 at station: 106
No weather data for 12/24/2018 at station: 106
No weather data for 12/25/2018 at station: 106
No weather data fo

No weather data for 9/26/2018 at station: 52
No weather data for 6/23/2018 at station: 52
No weather data for 6/24/2018 at station: 52
No weather data for 6/30/2018 at station: 52
No weather data for 7/5/2018 at station: 52
No weather data for 7/6/2018 at station: 52
No weather data for 7/7/2018 at station: 52
No weather data for 7/8/2018 at station: 52
No weather data for 7/9/2018 at station: 52
No weather data for 7/10/2018 at station: 52
No weather data for 7/13/2018 at station: 52
No weather data for 7/18/2018 at station: 52
No weather data for 7/19/2018 at station: 52
No weather data for 7/20/2018 at station: 52
No weather data for 7/21/2018 at station: 52
No weather data for 7/22/2018 at station: 52
No weather data for 7/23/2018 at station: 52
No weather data for 7/24/2018 at station: 52
No weather data for 7/25/2018 at station: 52
No weather data for 7/26/2018 at station: 52
No weather data for 7/29/2018 at station: 52
No weather data for 7/31/2018 at station: 52
No weather data

No weather data for 9/11/2018 at station: 52
No weather data for 9/15/2018 at station: 52
No weather data for 9/16/2018 at station: 52
No weather data for 9/20/2018 at station: 52
No weather data for 9/26/2018 at station: 52
No weather data for 7/6/2018 at station: 78
No weather data for 7/7/2018 at station: 78
No weather data for 7/6/2018 at station: 78
No weather data for 7/7/2018 at station: 78
No weather data for 7/6/2018 at station: 78
No weather data for 7/7/2018 at station: 78
No weather data for 7/6/2018 at station: 78
No weather data for 7/7/2018 at station: 78
No weather data for 7/8/2018 at station: 126
No weather data for 7/9/2018 at station: 126
No weather data for 8/6/2018 at station: 126
No weather data for 11/11/2018 at station: 126
No weather data for 7/8/2018 at station: 126
No weather data for 7/9/2018 at station: 126
No weather data for 8/6/2018 at station: 126
No weather data for 11/11/2018 at station: 126
No weather data for 7/8/2018 at station: 126
No weather dat

No weather data for 6/17/2018 at station: 147
No weather data for 8/9/2018 at station: 147
No weather data for 6/17/2018 at station: 147
No weather data for 8/9/2018 at station: 147
No weather data for 8/18/2018 at station: 5
No weather data for 8/19/2018 at station: 5
No weather data for 8/20/2018 at station: 5
No weather data for 8/6/2018 at station: 126
No weather data for 11/11/2018 at station: 126
No weather data for 8/6/2018 at station: 126
No weather data for 11/11/2018 at station: 126
No weather data for 8/6/2018 at station: 126
No weather data for 11/11/2018 at station: 126
No weather data for 6/17/2018 at station: 147
No weather data for 8/9/2018 at station: 147
No weather data for 6/17/2018 at station: 147
No weather data for 8/9/2018 at station: 147
No weather data for 6/17/2018 at station: 147
No weather data for 8/9/2018 at station: 147
No weather data for 6/17/2018 at station: 147
No weather data for 8/9/2018 at station: 147
No weather data for 3/31/2018 at station: 64
N

No weather data for 8/9/2018 at station: 171
No weather data for 10/16/2018 at station: 171
No weather data for 11/15/2018 at station: 171
No weather data for 11/16/2018 at station: 171
No weather data for 11/17/2018 at station: 171
No weather data for 11/18/2018 at station: 171
No weather data for 11/19/2018 at station: 171
No weather data for 11/20/2018 at station: 171
No weather data for 11/22/2018 at station: 171
No weather data for 11/23/2018 at station: 171
No weather data for 11/24/2018 at station: 171
No weather data for 7/21/2018 at station: 52
No weather data for 7/22/2018 at station: 52
No weather data for 7/23/2018 at station: 52
No weather data for 7/24/2018 at station: 52
No weather data for 7/25/2018 at station: 52
No weather data for 7/26/2018 at station: 52
No weather data for 7/29/2018 at station: 52
No weather data for 7/31/2018 at station: 52
No weather data for 8/1/2018 at station: 52
No weather data for 8/2/2018 at station: 52
No weather data for 8/4/2018 at stati

No weather data for 6/5/2018 at station: 83
No weather data for 6/6/2018 at station: 83
No weather data for 6/7/2018 at station: 83
No weather data for 6/8/2018 at station: 83
No weather data for 6/9/2018 at station: 83
No weather data for 6/10/2018 at station: 83
No weather data for 6/11/2018 at station: 83
No weather data for 6/12/2018 at station: 83
No weather data for 6/13/2018 at station: 83
No weather data for 6/14/2018 at station: 83
No weather data for 6/15/2018 at station: 83
No weather data for 6/16/2018 at station: 83
No weather data for 6/17/2018 at station: 83
No weather data for 7/24/2018 at station: 83
No weather data for 2/9/2018 at station: 64
No weather data for 2/10/2018 at station: 64
No weather data for 2/11/2018 at station: 64
No weather data for 3/31/2018 at station: 64
No weather data for 7/8/2018 at station: 64
No weather data for 8/6/2018 at station: 64
No weather data for 8/10/2018 at station: 64
No weather data for 10/30/2019 at station: 152
No weather data 

In [183]:
CA_fire_actual_DF = pd.DataFrame(CA_fire_actual_DF)
CA_fire_actual_DF.head(3)

,AcresBurned,ArchiveYear,CalFireIncident,County,Extinguished,Started,Stn Id,Precip (in),CIMIS Region,Days Burned,Precipitation,Temperature,Humidity,Wind Speed
0,410203.0,2018,True,Colusa,2019-01-04T09:33:00Z,2018-07-27T12:05:00Z,250.0,0.0,Sacramento Valley,162,0.019444,60.873457,64.654321,4.472840
1,48920.0,2018,True,Colusa,2019-01-04T09:18:00Z,2018-07-27T13:01:00Z,250.0,0.0,Sacramento Valley,162,0.019444,60.873457,64.654321,4.472840
2,75.0,2018,True,Colusa,2019-01-04T09:56:00Z,2018-07-04T13:14:00Z,250.0,0.0,Sacramento Valley,185,0.017213,62.727869,64.683060,4.425137


In [184]:
#drop the uselss feature first
CA_fire_actual_DF = CA_fire_actual_DF.drop(["Extinguished","Started"], axis=1)

In [190]:
from sklearn.preprocessing import  OneHotEncoder
def Encode_Dataframe(df, labels, enc):
    
    new_df = {}
    LeiLei_df = {}
    
    for i in labels:
        new_df[i] = df[i]
    new_df = pd.DataFrame(new_df)
    ans = enc.fit_transform(new_df)
    
    Names = new_df[i].unique()
    
    for i in Names:
        LeiLei_df[i] = np.reshape(enc.transform([[i]]), (-1,))
    
    for i in labels:
        df = df.drop(i, axis = 1)
    
    
    columns = []
    for i in range(ans.shape[1]):
        columns.append("{} {}".format(labels[0], i))
                       
    new_df = pd.DataFrame(ans, columns= columns )
    for i in new_df:
        df[i] = new_df[i]
    return df, LeiLei_df


label_feature = ["CalFireIncident","County", "Stn Id", "CIMIS Region"]

df = CA_fire_actual_DF.copy()
LeiLei_df =[]
for i in label_feature:
    enc = OneHotEncoder(sparse = False) 
    label = [i]
    df,abc = Encode_Dataframe(df, label, enc)
    LeiLei_df.append(abc)



In [191]:
for i in range(len(LeiLei_df)):
    for j in LeiLei_df[i]:
        temp = ""
        for k in LeiLei_df[i][j]:
            temp = temp+str(int(k))
        LeiLei_df[i][j] = temp


for i in range(len(LeiLei_df)):
    LeiLei_df[i] = dict(sorted(LeiLei_df[i].items(), key=lambda item: item[1], reverse=True))
LeiLei_df


[{False: '10', True: '01'},
 {'Alameda': '100000000000000000000000000000000000000000',
  'Amador': '010000000000000000000000000000000000000000',
  'Butte': '001000000000000000000000000000000000000000',
  'Colusa': '000100000000000000000000000000000000000000',
  'Contra Costa': '000010000000000000000000000000000000000000',
  'El Dorado': '000001000000000000000000000000000000000000',
  'Fresno': '000000100000000000000000000000000000000000',
  'Inyo': '000000010000000000000000000000000000000000',
  'Kern': '000000001000000000000000000000000000000000',
  'Kings': '000000000100000000000000000000000000000000',
  'Lassen': '000000000010000000000000000000000000000000',
  'Los Angeles': '000000000001000000000000000000000000000000',
  'Marin': '000000000000100000000000000000000000000000',
  'Mendocino': '000000000000010000000000000000000000000000',
  'Merced': '000000000000001000000000000000000000000000',
  'Modoc': '000000000000000100000000000000000000000000',
  'Monterey': '0000000000000000100

In [192]:
from sklearn import preprocessing

names = [  'ArchiveYear',   'Precip (in)',   'Days Burned',
            'Precipitation',   'Temperature',      'Humidity',    'Wind Speed',]
Nomalized_df = {}
for i in names:
    Nomalized_df[i] = df[i]
    print("name = ", i)
    print("max = ",max(df[i]), "min = ", min(df[i]))

scaler = preprocessing.MinMaxScaler()                  
Nomalized_df = scaler.fit_transform(pd.DataFrame(Nomalized_df))
Nomalized_df = pd.DataFrame(Nomalized_df, columns=names)
Nomalized_df.head(3)

name =  ArchiveYear
max =  2019 min =  2018
name =  Precip (in)
max =  0.35 min =  0.0
name =  Days Burned
max =  361 min =  1
name =  Precipitation
max =  0.258667 min =  0.0
name =  Temperature
max =  87.7 min =  0.0
name =  Humidity
max =  84.0 min =  0.0
name =  Wind Speed
max =  15.6 min =  0.0


,ArchiveYear,Precip (in),Days Burned,Precipitation,Temperature,Humidity,Wind Speed
0,0.0,0.0,0.447222,0.075170,0.694110,0.769694,0.286721
1,0.0,0.0,0.447222,0.075170,0.694110,0.769694,0.286721
2,0.0,0.0,0.511111,0.066545,0.715255,0.770036,0.283663


In [188]:
for i in names:
    df[i] = Nomalized_df[i]
df.head(3)

,AcresBurned,ArchiveYear,Precip (in),Days Burned,Precipitation,Temperature,Humidity,Wind Speed,CalFireIncident 0,CalFireIncident 1,...,CIMIS Region 2,CIMIS Region 3,CIMIS Region 4,CIMIS Region 5,CIMIS Region 6,CIMIS Region 7,CIMIS Region 8,CIMIS Region 9,CIMIS Region 10,CIMIS Region 11
0,410203.0,0.0,0.0,0.447222,0.075170,0.694110,0.769694,0.286721,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,48920.0,0.0,0.0,0.447222,0.075170,0.694110,0.769694,0.286721,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,75.0,0.0,0.0,0.511111,0.066545,0.715255,0.770036,0.283663,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [189]:
df.to_csv('data/Ca_fire_weather.csv')